In [6]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings

# Modelling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [8]:
df = pd.read_csv('data/stud.csv')

df['total score'] = df['math score'] + df['reading score'] + df['writing score']
df['average'] = df['total score']/3

X = df.loc[:, ['gender', 'race/ethnicity', 'lunch', 'test preparation course', 'reading score', 'writing score']]
y = df['math score']

In [9]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

X = preprocessor.fit_transform(X)

In [10]:
# separate into train/test

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((800, 13), (200, 13))

In [11]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [16]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}


eval_list = []

for model_name, model in models.items():
    model.fit(X_train, y_train)

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    eval_list.append((model_name, *evaluate_model(y_test, y_test_pred)))

out = pd.DataFrame(eval_list, columns = ['model_name', 'mae', 'rmse', 'r2']).sort_values(by='r2', ascending=False)
print(out)

                model_name       mae      rmse        r2
2                    Ridge  4.196159  5.388689  0.880668
0        Linear Regression  4.282500  5.488126  0.876224
8       AdaBoost Regressor  4.591658  5.896516  0.857117
7    CatBoosting Regressor  4.506695  5.986143  0.852740
5  Random Forest Regressor  4.692784  6.093967  0.847388
1                    Lasso  5.157882  6.519695  0.825320
6             XGBRegressor  5.136038  6.566012  0.822829
3    K-Neighbors Regressor  5.492000  6.915143  0.803487
4            Decision Tree  6.320000  8.003905  0.736735
